# Classification 10/20/22

Goal: Start classifying tidydata_1020222 using sktime module, specificaly classify based on differences in temp / rh 



In [59]:
import json
import datetime
import pandas as pd
import numpy as np
import sys
import os
import plotly.express as px
from sktime.classification.kernel_based import RocketClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sktime.classification.hybrid import HIVECOTEV2

In [3]:
df = pd.read_csv("../constants/td_ambient_102022.csv", )
df

,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
0,2022-07-20 07:20:00,23.189,51.268,0,14.333333,30.555556,1
1,2022-07-20 07:20:30,23.189,51.362,0,14.333333,30.555556,1
2,2022-07-20 07:21:00,23.189,51.488,0,14.333333,30.555556,1
3,2022-07-20 07:21:30,23.189,51.457,0,14.333333,30.555556,1
4,2022-07-20 07:22:00,23.165,51.517,0,14.333333,30.555556,1
...,...,...,...,...,...,...,...
123601,2022-09-11 07:58:00,29.196,50.861,1,19.611111,30.000000,0
123602,2022-09-11 07:58:30,29.196,50.861,1,19.611111,30.000000,0
123603,2022-09-11 07:59:00,29.196,50.796,1,19.611111,30.000000,0
123604,2022-09-11 07:59:30,29.196,50.796,1,19.611111,30.000000,0


In [28]:
# add temperature and rh deltas for each observation 
df["T_Delta"] =  df["Temp C"] - df["Ambient Temp"]
df["RH_Delta"] =  df["RH %"]- df["Ambient RH"]


In [34]:
df.head()

,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open,T_Delta,RH_Delta
0,2022-07-20 07:20:00,23.189,51.268,0,14.333333,30.555556,1,8.855667,20.712444
1,2022-07-20 07:20:30,23.189,51.362,0,14.333333,30.555556,1,8.855667,20.806444
2,2022-07-20 07:21:00,23.189,51.488,0,14.333333,30.555556,1,8.855667,20.932444
3,2022-07-20 07:21:30,23.189,51.457,0,14.333333,30.555556,1,8.855667,20.901444
4,2022-07-20 07:22:00,23.165,51.517,0,14.333333,30.555556,1,8.831667,20.961444


# Grouping and Stats

In [30]:
# do some grouping 

# group by window open vs window closed -> where are the deltas greater 
df.drop("DateTime", axis=1).groupby(["Window Open"]).agg(["mean", "std"]).loc[:,["T_Delta", "RH_Delta"]]

T_Delta             RH_Delta          
                 mean       std       mean       std
Window Open                                         
0            5.752171  4.206669  30.640394  8.547424
1            6.257008  3.594757  28.279236  7.846737

In [32]:
# group by rooms -> where are the deltas greater 
df.drop("DateTime", axis=1).groupby(["Room"]).agg(["mean", "std"]).loc[:,["T_Delta", "RH_Delta"]]

T_Delta             RH_Delta          
          mean       std       mean       std
Room                                         
0     5.729435  3.780207  29.178837  8.143704
1     6.301897  4.010265  29.637181  8.395411

In [33]:
# group by window open vs window closed and rooms -> where are the deltas greater 
df.drop("DateTime", axis=1).groupby(["Window Open", "Room"]).agg(["mean", "std"]).loc[:,["T_Delta", "RH_Delta"]]

T_Delta             RH_Delta          
                      mean       std       mean       std
Window Open Room                                         
0           0     1.603624  4.786734  37.193811  7.443892
            1     6.059046  3.992906  30.155626  8.423356
1           0     6.020292  3.521054  28.613806  7.889349
            1     8.272099  3.584829  25.431145  6.841461

# Attempt at classification of hourly data 

In [39]:
times = pd.to_datetime(df.DateTime)

In [49]:
# aggregate data based on hour
df_hour = df.groupby([times.dt.date, times.dt.hour, "Room"]).mean()
df_hour.head()

Temp C       RH %  Ambient Temp  Ambient RH  \
DateTime   DateTime Room                                                   
2022-07-20 7        0     23.326400  51.622238     14.333333   30.555556   
                    1     23.422100  52.867775     14.333333   30.555556   
           8        0     23.736167  50.783575     14.666667   30.555556   
                    1     23.740725  52.640442     14.666667   30.555556   
           9        0     24.307792  50.038683     15.388889   28.888889   

                          Window Open   T_Delta   RH_Delta  
DateTime   DateTime Room                                    
2022-07-20 7        0          1.0000  8.993067  21.066682  
                    1          0.2375  9.088767  22.312219  
           8        0          1.0000  9.069500  20.228019  
                    1          1.0000  9.074058  22.084886  
           9        0          1.0000  8.918903  21.149794

In [ ]:
y_df = df.groupby([times.dt.date, times.dt.hour]).mean()
df_hour.head(24)

## based on T_Delta
can we classify whether the window is opened or closed based on the magnitude of T_Delta alone?

In [61]:
x = df_hour.loc[:, ["T_Delta", "RH_Delta"]] # seems like needs to be 2d? panel??/
x

T_Delta   RH_Delta
DateTime   DateTime Room                     
2022-07-20 7        0     8.993067  21.066682
                    1     9.088767  22.312219
           8        0     9.069500  20.228019
                    1     9.074058  22.084886
           9        0     8.918903  21.149794
...                            ...        ...
2022-09-11 6        1     9.763292  20.767308
           7        0     9.188531  21.962300
                    1     9.619472  20.809808
           8        0     8.864667  21.629000
                    1     9.362667  20.796000

[1036 rows x 2 columns]

In [51]:
y = df_hour.loc[:, "Window Open"]
y

DateTime    DateTime  Room
2022-07-20  7         0       1.0000
                      1       0.2375
            8         0       1.0000
                      1       1.0000
            9         0       1.0000
                               ...  
2022-09-11  6         1       0.0000
            7         0       1.0000
                      1       0.0000
            8         0       1.0000
                      1       0.0000
Name: Window Open, Length: 1036, dtype: float64

In [62]:
# convert to numpy 
xnp = x.to_numpy()
ynp = y.to_numpy()

In [63]:
# split data into train and test 
X_train, X_test, y_train, y_test = train_test_split(xnp, ynp,test_size=0.2, random_state=42)

In [64]:
hc2 = HIVECOTEV2(time_limit_in_minutes=1)
hc2.fit(X_train, y_train)
y_pred = hc2.predict(X_test)
accuracy_score(y_test, y_pred)

ValueError: high <= 0

In [65]:
# # sktime classifier
rocket = RocketClassifier()
rocket.fit(X_train, y_train)
y_pred = rocket.predict(X_test)
accuracy_score(y_test, y_pred)

ValueError: Unknown label type: (array([0.35833333, 0.        , 1.        , 1.        , 0.        ,
       1.        , 1.        , 0.        , 0.        , 1.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 1.        , 0.        , 1.        ,
       1.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 1.        , 1.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 1.        ,
       1.        , 1.        , 0.        , 1.        , 0.        ,
       1.        , 0.        , 1.        , 0.        , 1.        ,
       0.        , 1.        , 1.        , 0.89166667, 0.        ,
       0.675     , 1.        , 0.        , 1.        , 0.        ,
       0.        , 1.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       1.        , 0.        , 1.        , 1.        , 1.        ,
       1.        , 0.        

# Split into train / test data

In [32]:
x = df.iloc[:, 1:4]
x

,Temp C,RH %,Room
0,23.189,51.268,0
1,23.189,51.362,0
2,23.189,51.488,0
3,23.189,51.457,0
4,23.165,51.517,0
...,...,...,...
123601,29.196,50.861,1
123602,29.196,50.861,1
123603,29.196,50.796,1
123604,29.196,50.796,1


In [33]:
x = x.to_numpy()
x

array([[23.189, 51.268,  0.   ],
       [23.189, 51.362,  0.   ],
       [23.189, 51.488,  0.   ],
       ...,
       [29.196, 50.796,  1.   ],
       [29.196, 50.796,  1.   ],
       [29.196, 50.796,  1.   ]])

In [34]:
y = df["Window Open"]
y = y.to_numpy()
y

array([1, 1, 1, ..., 0, 0, 0])

In [35]:

X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=42)

In [36]:
X_train

array([[24.731, 52.678,  1.   ],
       [29.897, 48.431,  1.   ],
       [24.103, 57.157,  1.   ],
       ...,
       [24.997, 49.381,  0.   ],
       [25.288, 51.742,  1.   ],
       [29.872, 49.011,  1.   ]])

In [37]:
y_test

array([0, 0, 0, ..., 1, 0, 0])

In [38]:
# data tooo large -> 82 mins running and still not done 
# rocket = RocketClassifier()
# rocket.fit(X_train, y_train)
# y_pred = rocket.predict(X_test)
# accuracy_score(y_test, y_pred)

# try to sample the data instead....